# Utilities

### Metrics calculation

1. Set dataset path and scale applied to spines during visualization.

In [3]:
from notebook_widgets import SpineMeshDataset

# set dataset path and scale applied to spines during visualization
dataset_path = "example dataset"
scale = (1, 1, 1)

# load and scale spine mesh dataset
spine_dataset = SpineMeshDataset().load(dataset_path)
scaled_spine_dataset = SpineMeshDataset().load(dataset_path)
scaled_spine_dataset.apply_scale(scale)

2. View Dendrite Skeleton.

In [4]:
from notebook_widgets import view_skeleton_widget


display(view_skeleton_widget(scaled_spine_dataset))

interactive(children=(Dropdown(description='Dendrite:', options=('9009\\Ab\\ab18-1\\surface_mesh.off', '9009\\…

3. View Dendrite segmentation.

In [5]:
from notebook_widgets import dendrite_segmentation_view_widget

display(dendrite_segmentation_view_widget(scaled_spine_dataset))

interactive(children=(Dropdown(description='Dendrite:', options=('9009\\Ab\\ab18-1\\surface_mesh.off', '9009\\…

4. View Chords.

Set `num_of_chords` and `num_of_bins`. Histograms can be exported to `dataset_path/chords_%num_of_chords%_chords_%num_of_bins%_bins.csv` file. Histograms are only exported for meshes that were viewed! Otherwise it would take too much time to calculate. Use `Calculate Metrics` cell to calculate metrics for the entire dataset.

In [6]:
from notebook_widgets import spine_chords_widget


num_of_chords = 30000
num_of_bins = 100

display(spine_chords_widget(spine_dataset, scaled_spine_dataset, dataset_path, num_of_chords, num_of_bins))

5. Calculate Metrics.

Metrics for the dataset will be saved to `dataset_path/metrics.csv`. Chords histograms will be saved separately to `dataset_path/chords.csv`. 

In [7]:
from spine_metrics import SpineMetricDataset


# chord method parameters
num_of_chords = 30000
num_of_bins = 100

# calculate metrics
metric_names = ["ChordDistribution", "OpenAngle", "CVD", "AverageDistance",
                "LengthVolumeRatio", "LengthAreaRatio", "JunctionArea",
                "Length", "Area", "Volume", "ConvexHullVolume", "ConvexHullRatio"]
metric_params = [{"num_of_chords": num_of_chords, "num_of_bins": num_of_bins},
                 {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}]
spine_metrics = SpineMetricDataset()
spine_metrics.calculate_metrics(spine_dataset.spine_meshes, metric_names, metric_params)
spine_metrics.save(f"{dataset_path}/metrics.csv")    

spine_metrics.get_metrics_subset(["ChordDistribution"]).save_as_array(f"{dataset_path}/chords.csv")

6. View Spines in Dataset.

In [ ]:
from spine_metrics import SpineMetricDataset
from notebook_widgets import spine_dataset_view_widget


metrics = SpineMetricDataset().load(f"{dataset_path}/metrics.csv")
display(spine_dataset_view_widget(scaled_spine_dataset, metrics))

### Utilities for classifiaction and clustering

7. Merge Manual Classifications.

In [ ]:
from spine_metrics import SpineMetricDataset
from notebook_widgets import create_dir, remove_file
from spine_fitter import SpineGrouping
import pathlib

dataset_path = "example dataset"
merged_path = f"{dataset_path}/manual_classification/manual_classification_merged.json"
merged_reduced_path = f"{dataset_path}/manual_classification/manual_classification_merged_reduced.json"

# remove old merged classifications
remove_file(merged_path)
remove_file(merged_reduced_path)

# load manual classifications
path = pathlib.Path(f"{dataset_path}/manual_classification")
classification_paths = [str(classification_path) for classification_path in path.glob("*.json")]
groupings = [SpineGrouping().load(path) for path in classification_paths]
print(f"Merging manual classifications: {classification_paths}\n")

# merge classifications
merged_grouping = SpineGrouping.merge(groupings, outliers_label="Unclassified")
merged_grouping.save(merged_path)
print(f"Saved to \"{merged_path}\".")

# remove spines with no consensus, filopodias and labeled-as-outliers
contested = SpineGrouping.get_contested_samples(groupings)
merged_grouping.remove_samples(contested)
merged_grouping.remove_samples(merged_grouping.groups["Filopodia"].copy())
merged_grouping.remove_samples(merged_grouping.groups["Outlier"].copy())
del merged_grouping.groups["Filopodia"]
del merged_grouping.groups["Outlier"]

merged_grouping.save(merged_reduced_path)
print(f"Saved to \"{merged_reduced_path}\".")

8. Consensus Table.

In [ ]:
from notebook_widgets import consensus_widget
from spine_fitter import SpineGrouping
import pathlib

# load manual classifications
path = pathlib.Path(f"{dataset_path}/manual_classification")
classification_paths = path.glob("manual_classification_?.json")
groupings = [SpineGrouping().load(str(path)) for path in classification_paths]

# show consensus table
display(consensus_widget(groupings))

9. Generate group labeling from dataset paths hierarchy.

In [2]:
from pathlib import Path
from collections import defaultdict
import re

from spine_fitter import SpineGrouping

dataset_path = " "
path_pattern = "*/*/spine_*.off"
label_patern = "(?P<c>.*)/.*/spine_.*.off"
output_subpath = "labeling_by_dirs.json"

path = Path(dataset_path)
labeling = defaultdict(set)

for l in list(path.glob(path_pattern)):
    m = re.match(f"{dataset_path}/{label_patern}", l.as_posix())
    if m:
        labeling[m.group('c')].add(str(l))

result_grouping = SpineGrouping(groups=labeling)
result_grouping.save(filename=f"{dataset_path}/{output_subpath}")